# Lexicon Score based on BoE Wordlist

### 1. Import the data

In [2]:
import pandas as pd
import re                                               # Regular expression operations on strings.
from collections import defaultdict                     # Creating dictionaries that return default value for nonexistent keys.
from nltk.corpus import wordnet as wn                   # Lexical database for retrieving word relationships & meanings.
from nltk.stem import WordNetLemmatizer, PorterStemmer  # Reducing words to base or root form.
import contractions                                     # Expanding/contracting text contractions.
from nltk.corpus import stopwords                       # Providing list of common words to exclude from analysis.
from nltk import word_tokenize, pos_tag                 # Splitting text into words and tags with part of speech
from nltk.stem import WordNetLemmatizer, PorterStemmer  # Reducing words to base or root form.

In [4]:
# Load the CSV file as speeches
speeches = pd.read_csv('/Users/kaferrante/Documents/Python/_Course4_Project/all_speeches.csv')

# View the DataFrame
speeches.head()

,reference,country,date,title,author,is_gov,text
0,r901128a_BOA,australia,1990-11-28,A Proper Role for Monetary Policy,fraser,0,They would no doubt argue that to have two obj...
1,r911003a_BOA,australia,1991-10-03,,fraser,0,Today I wish to talk about real interest rates...
2,r920314a_BOA,australia,1992-03-14,,fraser,0,I welcome this opportunity to talk about prosp...
3,r920529a_BOA,australia,1992-05-29,,fraser,0,It is a pleasure to have this opportunity to a...
4,r920817a_BOA,australia,1992-08-17,,fraser,0,"As a long-time fan of Don Sanders, I am deligh..."


In [5]:
# Load the Excel file of BoE sentiment labelled wordlist
sentiment_lexicon = pd.read_excel('/Users/kaferrante/Documents/Python/_Course4_Project/sentiment_labelled_wordlist.xlsx')

# View the DataFrame
sentiment_lexicon.head()

,Word,Negative,Positive,Uncertainty,Litigious,Strong,Weak,Constraining
0,ABANDON,1,0,0,0,0,0,0
1,ABANDONED,1,0,0,0,0,0,0
2,ABANDONING,1,0,0,0,0,0,0
3,ABANDONMENT,1,0,0,0,0,0,0
4,ABANDONMENTS,1,0,0,0,0,0,0


In [8]:
# Rename columns
sentiment_lexicon = sentiment_lexicon.rename(columns={
    'Negative': 'negative',
    'Positive': 'positive',
    'Uncertainty': 'uncertainty',
    'Litigious': 'litigious',
    'Strong': 'strong',
    'Weak': 'weak',
    'Constraining': 'constraining',
})

# View the DataFrame
sentiment_lexicon.head()

,Word,negative,positive,uncertainty,litigious,strong,weak,constraining
0,ABANDON,1,0,0,0,0,0,0
1,ABANDONED,1,0,0,0,0,0,0
2,ABANDONING,1,0,0,0,0,0,0
3,ABANDONMENT,1,0,0,0,0,0,0
4,ABANDONMENTS,1,0,0,0,0,0,0


### 2. Clean the data

**Text cleaned** by
- Removing unwanted characters
- Removing any special characters and punctuation
- Converting text to lower case
- Removing stopwords
- tokenisation: Split the cleaned text into individual words, so that text can be analysed at word level.
- Lemmatisation: Reduce words to its base or dictionary form (the lemma).

**Text cleaning functions**

In [10]:
# Preprocessing function
def preprocess_text(text):
    text = contractions.fix(text)  # Expand contractions i.e I'm not good goes to I am not good
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub('#', '', text)         # Remove hashtags
    text = re.sub(r'\W', ' ', text)      # Remove special characters
    text = text.lower()                  # Convert to lowercase
    #Below is to create a set of stop words from the NLTK library's predefined list but not is excluded.
    stop_words = set(stopwords.words('english')) - {'not'} 
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [12]:
# Define the tag map for POS tagging.
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

# Lemmatise the tokens with correct POS tags.
lemma_function = WordNetLemmatizer()

# Lemmatisation function.
def lemmatize_tokens(tokens):
    #For each word in the token list, it lemmatizes the word with the correct part-of-speech
    lemmatized_tokens = [lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(tokens)]
    return lemmatized_tokens

**Apply text cleaning functions**

In [14]:
# Apply the preprocessing function
speeches['text_cleaned'] = speeches['text'].apply(preprocess_text)

In [15]:
# Apply the tokenisation function
speeches['text_tokenised'] = speeches['text_cleaned'].apply(word_tokenize)

In [16]:
# Apply the lemmatisation function
speeches['text_lemmatised'] = speeches['text_tokenised'].apply(lemmatize_tokens)

In [17]:
# Convert list of words into a string
speeches['text_lemmatised_str'] = speeches['text_lemmatised'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [18]:
# Count the number of words in original text
speeches['word_count_text'] = speeches['text'].str.split().apply(len)

In [19]:
# Count the number of words in lemmatised text
speeches['word_count_text_cleaned'] = speeches['text_lemmatised_str'].str.split().apply(len)

In [20]:
# View the Dataframe
speeches.head()

,reference,country,date,title,author,is_gov,text,text_cleaned,text_tokenised,text_lemmatised,text_lemmatised_str,word_count_text,word_count_text_cleaned
0,r901128a_BOA,australia,1990-11-28,A Proper Role for Monetary Policy,fraser,0,They would no doubt argue that to have two obj...,would doubt argue two objectives like trying c...,"[would, doubt, argue, two, objectives, like, t...","[would, doubt, argue, two, objective, like, tr...",would doubt argue two objective like try cake ...,3627,1919
1,r911003a_BOA,australia,1991-10-03,,fraser,0,Today I wish to talk about real interest rates...,today wish talk real interest rates mainly his...,"[today, wish, talk, real, interest, rates, mai...","[today, wish, talk, real, interest, rate, main...",today wish talk real interest rate mainly hist...,3054,1754
2,r920314a_BOA,australia,1992-03-14,,fraser,0,I welcome this opportunity to talk about prosp...,welcome opportunity talk prospects banks austr...,"[welcome, opportunity, talk, prospects, banks,...","[welcome, opportunity, talk, prospect, bank, a...",welcome opportunity talk prospect bank austral...,3399,1867
3,r920529a_BOA,australia,1992-05-29,,fraser,0,It is a pleasure to have this opportunity to a...,pleasure opportunity address influential gathe...,"[pleasure, opportunity, address, influential, ...","[pleasure, opportunity, address, influential, ...",pleasure opportunity address influential gathe...,3841,2123
4,r920817a_BOA,australia,1992-08-17,,fraser,0,"As a long-time fan of Don Sanders, I am deligh...",long time fan sanders delighted participating ...,"[long, time, fan, sanders, delighted, particip...","[long, time, fan, sander, delight, participate...",long time fan sander delight participate tribu...,4152,2150


### 3. Calculating simple sentiment score with BoE Wordlist

**Sentiment score** is calculated as follows:
- positive and negative words in each speech are counted
- sentiment score = psotive word count - negative word count /  wordcount

In [31]:
# Prepare the lexicon
sentiment_lexicon = sentiment_lexicon.copy()

# Define categories
categories = [
     'negative',
     'positive',
     'uncertainty',
     'litigious',
     'strong',
     'weak',
     'constraining',
 ]

# Create dictionary of categories, containing words that belong to that category based on your sentiment lexicon.
word_sets = {
    cat: set(sentiment_lexicon.loc[sentiment_lexicon[cat] == 1, 'Word'].str.lower())
    for cat in categories
}

In [33]:
# Define function to apply the lexicon to the tokens
def lexicon_counts(tokens):
    return pd.Series({
        cat: sum(t in word_sets[cat] for t in tokens)
        for cat in categories
    })

# Apply the lexicon_counts function to 'text_lemmatised' column
category_counts = speeches['text_lemmatised'].apply(lexicon_counts)

# Concatenate category counts to the original DataFrame
speeches = pd.concat([speeches, category_counts], axis=1)

# Calculate total sentiment words across all categories for each speech
speeches['word_count_sentiment'] = category_counts.sum(axis=1)

# View the DataFrame
speeches.head()

,reference,country,date,title,author,is_gov,text,text_cleaned,text_tokenised,text_lemmatised,...,word_count_text,word_count_text_cleaned,negative,positive,uncertainty,litigious,strong,weak,constraining,word_count_sentiment
0,r901128a_BOA,australia,1990-11-28,A Proper Role for Monetary Policy,fraser,0,They would no doubt argue that to have two obj...,would doubt argue two objectives like trying c...,"[would, doubt, argue, two, objectives, like, t...","[would, doubt, argue, two, objective, like, tr...",...,3627,1919,84,58,32,5,10,15,13,217
1,r911003a_BOA,australia,1991-10-03,,fraser,0,Today I wish to talk about real interest rates...,today wish talk real interest rates mainly his...,"[today, wish, talk, real, interest, rates, mai...","[today, wish, talk, real, interest, rate, main...",...,3054,1754,53,28,35,2,3,16,12,149
2,r920314a_BOA,australia,1992-03-14,,fraser,0,I welcome this opportunity to talk about prosp...,welcome opportunity talk prospects banks austr...,"[welcome, opportunity, talk, prospects, banks,...","[welcome, opportunity, talk, prospect, bank, a...",...,3399,1867,43,67,33,8,11,16,13,191
3,r920529a_BOA,australia,1992-05-29,,fraser,0,It is a pleasure to have this opportunity to a...,pleasure opportunity address influential gathe...,"[pleasure, opportunity, address, influential, ...","[pleasure, opportunity, address, influential, ...",...,3841,2123,62,56,43,6,7,20,8,202
4,r920817a_BOA,australia,1992-08-17,,fraser,0,"As a long-time fan of Don Sanders, I am deligh...",long time fan sanders delighted participating ...,"[long, time, fan, sanders, delighted, particip...","[long, time, fan, sander, delight, participate...",...,4152,2150,72,62,42,6,12,27,13,234


In [37]:
# Calculate the sentiment score by subtracting the negative score from the positive score
# abd dividing by the total number of words in the lemmatised text
speeches['sentiment_lexicon_simple'] = (speeches['positive'] - speeches['negative'])/ speeches['word_count_sentiment']

In [39]:
# View the dataframe
speeches.head()

,reference,country,date,title,author,is_gov,text,text_cleaned,text_tokenised,text_lemmatised,...,word_count_text_cleaned,negative,positive,uncertainty,litigious,strong,weak,constraining,word_count_sentiment,sentiment_lexicon_simple
0,r901128a_BOA,australia,1990-11-28,A Proper Role for Monetary Policy,fraser,0,They would no doubt argue that to have two obj...,would doubt argue two objectives like trying c...,"[would, doubt, argue, two, objectives, like, t...","[would, doubt, argue, two, objective, like, tr...",...,1919,84,58,32,5,10,15,13,217,-0.119816
1,r911003a_BOA,australia,1991-10-03,,fraser,0,Today I wish to talk about real interest rates...,today wish talk real interest rates mainly his...,"[today, wish, talk, real, interest, rates, mai...","[today, wish, talk, real, interest, rate, main...",...,1754,53,28,35,2,3,16,12,149,-0.167785
2,r920314a_BOA,australia,1992-03-14,,fraser,0,I welcome this opportunity to talk about prosp...,welcome opportunity talk prospects banks austr...,"[welcome, opportunity, talk, prospects, banks,...","[welcome, opportunity, talk, prospect, bank, a...",...,1867,43,67,33,8,11,16,13,191,0.125654
3,r920529a_BOA,australia,1992-05-29,,fraser,0,It is a pleasure to have this opportunity to a...,pleasure opportunity address influential gathe...,"[pleasure, opportunity, address, influential, ...","[pleasure, opportunity, address, influential, ...",...,2123,62,56,43,6,7,20,8,202,-0.029703
4,r920817a_BOA,australia,1992-08-17,,fraser,0,"As a long-time fan of Don Sanders, I am deligh...",long time fan sanders delighted participating ...,"[long, time, fan, sanders, delighted, particip...","[long, time, fan, sander, delight, participate...",...,2150,72,62,42,6,12,27,13,234,-0.042735


### 4. Calculate weighted sentiment score with BoE Wordlist

**Sentiment score** is calculated as follows:
- words in all categories in each speech are counted (positive, negative, uncertainty, litigious, strong, weak, constraining)
- weights are applied to each category to show how important the influence on the sentiment is
- sentiment score = category counts * category weights / wordcount

**Category weights** based on importance:
- Negative -1       Standard negative words
- Positive +1.5     Standard positive words - compensate for negatively skewed wordlist
- Uncertainty 0.2   Words expressing doubt or ambiguity, less impactful than outright negative or positive words
- Litigious -0.2    Words related to lawsuits or legal issues, potentially negative or impactful depending on context
- Strong +1.5       Words with high intensity or impact, thus given more weight
- Weak +0.5         Words with less impact, so given lesser weight than 'Strong' words
- Constraining -0.5 Words implying restriction or limitations, generally negative

In [45]:
# Assign weights to the categories
category_weights = {
    'negative': -1,
    'positive': 1.5,
    'uncertainty': 0.2,
    'litigious': -0.2,
    'strong': 1.5,
    'weak': 0.5,
    'constraining': -0.5
}

In [47]:
# Define function to apply the lexicon to the text
def lexicon_score_weighted(tokens):
    score = 0
    for cat in categories:
        count = sum(t in word_sets[cat] for t in tokens)
        score += count * category_weights[cat]
    return score

# Compute counts and store as a new column
speeches['sentiment_lexicon_weighted'] = speeches['text_lemmatised'].apply(lexicon_score_weighted) \
                                                    / speeches['word_count_sentiment']

# View the DataFrame
speeches.head()

,reference,country,date,title,author,is_gov,text,text_cleaned,text_tokenised,text_lemmatised,...,negative,positive,uncertainty,litigious,strong,weak,constraining,word_count_sentiment,sentiment_lexicon_simple,sentiment_lexicon_weighted
0,r901128a_BOA,australia,1990-11-28,A Proper Role for Monetary Policy,fraser,0,They would no doubt argue that to have two obj...,would doubt argue two objectives like trying c...,"[would, doubt, argue, two, objectives, like, t...","[would, doubt, argue, two, objective, like, tr...",...,84,58,32,5,10,15,13,217,-0.119816,0.112442
1,r911003a_BOA,australia,1991-10-03,,fraser,0,Today I wish to talk about real interest rates...,today wish talk real interest rates mainly his...,"[today, wish, talk, real, interest, rates, mai...","[today, wish, talk, real, interest, rate, main...",...,53,28,35,2,3,16,12,149,-0.167785,0.014094
2,r920314a_BOA,australia,1992-03-14,,fraser,0,I welcome this opportunity to talk about prosp...,welcome opportunity talk prospects banks austr...,"[welcome, opportunity, talk, prospects, banks,...","[welcome, opportunity, talk, prospect, bank, a...",...,43,67,33,8,11,16,13,191,0.125654,0.421466
3,r920529a_BOA,australia,1992-05-29,,fraser,0,It is a pleasure to have this opportunity to a...,pleasure opportunity address influential gathe...,"[pleasure, opportunity, address, influential, ...","[pleasure, opportunity, address, influential, ...",...,62,56,43,6,7,20,8,202,-0.029703,0.227228
4,r920817a_BOA,australia,1992-08-17,,fraser,0,"As a long-time fan of Don Sanders, I am deligh...",long time fan sanders delighted participating ...,"[long, time, fan, sanders, delighted, particip...","[long, time, fan, sander, delight, participate...",...,72,62,42,6,12,27,13,234,-0.042735,0.227350


### 5. Export the DataFrame

In [57]:
# Export the DataFrame to CSV
speeches.to_csv('speeches_sentiment.csv', index=False)

In [59]:
# Load the CSV file as speeches
speeches_test = pd.read_csv('/Users/kaferrante/Documents/Python/_Course4_Project/speeches_sentiment.csv')

# View the DataFrame
speeches_test.head()

,reference,country,date,title,author,is_gov,text,text_cleaned,text_tokenised,text_lemmatised,...,negative,positive,uncertainty,litigious,strong,weak,constraining,word_count_sentiment,sentiment_lexicon_simple,sentiment_lexicon_weighted
0,r901128a_BOA,australia,1990-11-28,A Proper Role for Monetary Policy,fraser,0,They would no doubt argue that to have two obj...,would doubt argue two objectives like trying c...,"['would', 'doubt', 'argue', 'two', 'objectives...","['would', 'doubt', 'argue', 'two', 'objective'...",...,84,58,32,5,10,15,13,217,-0.119816,0.112442
1,r911003a_BOA,australia,1991-10-03,,fraser,0,Today I wish to talk about real interest rates...,today wish talk real interest rates mainly his...,"['today', 'wish', 'talk', 'real', 'interest', ...","['today', 'wish', 'talk', 'real', 'interest', ...",...,53,28,35,2,3,16,12,149,-0.167785,0.014094
2,r920314a_BOA,australia,1992-03-14,,fraser,0,I welcome this opportunity to talk about prosp...,welcome opportunity talk prospects banks austr...,"['welcome', 'opportunity', 'talk', 'prospects'...","['welcome', 'opportunity', 'talk', 'prospect',...",...,43,67,33,8,11,16,13,191,0.125654,0.421466
3,r920529a_BOA,australia,1992-05-29,,fraser,0,It is a pleasure to have this opportunity to a...,pleasure opportunity address influential gathe...,"['pleasure', 'opportunity', 'address', 'influe...","['pleasure', 'opportunity', 'address', 'influe...",...,62,56,43,6,7,20,8,202,-0.029703,0.227228
4,r920817a_BOA,australia,1992-08-17,,fraser,0,"As a long-time fan of Don Sanders, I am deligh...",long time fan sanders delighted participating ...,"['long', 'time', 'fan', 'sanders', 'delighted'...","['long', 'time', 'fan', 'sander', 'delight', '...",...,72,62,42,6,12,27,13,234,-0.042735,0.227350


In [61]:
speeches_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7721 entries, 0 to 7720
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   reference                   7721 non-null   object 
 1   country                     7721 non-null   object 
 2   date                        7721 non-null   object 
 3   title                       7721 non-null   object 
 4   author                      7721 non-null   object 
 5   is_gov                      7721 non-null   int64  
 6   text                        7721 non-null   object 
 7   text_cleaned                7721 non-null   object 
 8   text_tokenised              7721 non-null   object 
 9   text_lemmatised             7721 non-null   object 
 10  text_lemmatised_str         7721 non-null   object 
 11  word_count_text             7721 non-null   int64  
 12  word_count_text_cleaned     7721 non-null   int64  
 13  negative                    7721 